## Alternative Lösung Aufgabe 3 - Breitensuche
**Autor: Maarten Jung**

Die Komplexität der Breitensuche ist letztendlich von der konkreten Implementierung des Enqueueings abhängig. Durch eine Optimierung des vorhandenen Codes, kann eine Reduzierung der Kompexität auf $\mathcal{O}(b^{d})$ erreicht werden. Dennoch wird auf die Definition aus der Vorlesung mit $\mathcal{O}(b^{d+1})$ verwiesen, die auch eine weniger optimierte Implementierung abdeckt.

In [0]:
import math
import sys
import random


def codes_with_routes(codes):
    """
    returns the routes for the given codes, specified in routes.dat
    :param codes: the codes for which routes should be extracted
    :return: set of codes with
    """
    route_codes = set([])
    for line in open("routes.dat", encoding='utf-8'):
        split = line.split(",")
        fr = split[2].strip("\"")
        to = split[4].strip("\"")
        if fr in codes and to in codes:
            route_codes.add(fr)
            route_codes.add(to)
    return route_codes


def codes_with_airport_info():
    """
    parse all codes from airports.dat that have all required airport information

    :return: set of codes with complete airport information
    """
    codes = set([])
    for line in open("airports.dat", encoding='utf-8'):
        split = line.split(",")
        if len(split) == 12:
            code = split[4].strip("\"")
            city = split[2].strip("\"")
            country = split[3].strip("\"")
            x = float(split[6])
            y = float(split[7])
            if city and country and x and y:
                codes.add(code)
    return codes


def distance(fr, to):
    """
    calculates the distance between the two airports fr and to
    e.g. http://www.koordinaten.de/informationen/formel.shtml
    :rtype: float
    :return the distance between fr and to
    """
    if fr == to:
        return 0.0
    try:
        return math.acos(
            math.sin(math.radians(fr.x)) * math.sin(math.radians(to.x)) +
            math.cos(math.radians(fr.x)) * math.cos(math.radians(to.x)) *
            math.cos(math.radians(to.y - fr.y))
        ) * 6378.137
    except ArithmeticError:
        print(fr.code + " " + to.code)
        sys.exit()


def print_path(airport_path):
    """
    prints a traveled path (airports) and calculates the distance
    :param airport_path: the path of airports
    """
    for a in airport_path:
        print(a)
    dist = 0.0
    for i in range(len(airport_path) - 1):
        dist += distance(airport_path[i], airport_path[i + 1])
    print("distance: %d km" % dist)


def suggest_next(fr):
    """
    random next airport based on the given airports connections
    :param fr: the airport
    :return: random next airport connected to the given one
    """
    fr_connections = list(fr.connections)
    return fr_connections[random.randint(0, len(fr_connections) - 1)]


def construct_path(parent, fr, to):
    """
    constructs the search path as list based on the parent relations during the search
    :rtype: list
    :return the path
    """
    path = [to]
    while path[-1] != fr:  # last element of path is not fr
        path.append(parent[path[-1]])
    path.reverse()  # path is in wrong order, so reverse
    return path


class Airport:
    """
    implements an airport with airport code, city, country, x, y, and connections
    """

    def __str__(self) -> str:
        return "%s, %s, %s" % (self.code, self.city, self.country)

    def __init__(self, code):
        """
        creates an airport object with the given code
        :param code: code of the airport
        """
        self.code = code
        self.connections = list([])
        self.city = ""
        self.country = ""
        self.x = 0.0
        self.y = 0.0

    def set(self, city, country, x, y):
        """
        sets the required information for this airport
        :param city: the city name
        :param country: the country name
        :param x: coordinate x longitude
        :param y: coordinate y latitude
        """
        self.city = city
        self.country = country
        self.x = x
        self.y = y

    def fly_to(self, to):
        """
        adds a connection for this airport
        :param to: the airport to which a connection exists
        """
        if to not in self.connections:
            self.connections.append(to)


class Airports:
    """
    implements a set of airports with their connections
    """

    def __init__(self):
        """
        creates the collection of airports by reading all data files
        """
        self.airports_by_codes = {}  # a dictionary with code as key and airport as value
        self.airports_by_cities = {}  # a dictionary with city as key and a set of its airports as value
        self.airports_by_countries = {}  # a dictionary with countries as key and a set of its airports as values

        # identify all good codes (contain all information and have at least one route) as first filter
        self.good_codes = codes_with_routes(codes_with_airport_info())

        # get remaining information for airports
        self.read_routes()
        self.read_airport_info()
        self.index_cities()
        self.index_countries()

    def get(self, code):
        """
        gets a single airport based on its code
        :param code: the code of the airport
        :return: the airport if already available, if not a new airport
        """
        if code not in self.airports_by_codes:
            a = Airport(code)
            self.airports_by_codes[code] = a
        return self.airports_by_codes[code]

    def read_routes(self):
        """
        reads the routes of airports from the routes.dat file
        """
        for line in open("routes.dat", encoding='utf-8'):
            split = line.split(",")
            fr = split[2].strip("\"")
            to = split[4].strip("\"")
            if fr in self.good_codes and to in self.good_codes:
                fr_airport = self.get(fr)
                to_airport = self.get(to)
                fr_airport.fly_to(to_airport)
                to_airport.fly_to(fr_airport)

    def read_airport_info(self):
        """
        reads the information of all airports from the airports.dat file
        """
        for line in open("airports.dat", encoding='utf-8'):
            split = line.split(",")
            if len(split) == 12:
                code = split[4].strip("\"")
                city = split[2].strip("\"")
                country = split[3].strip("\"")
                x = float(split[6])
                y = float(split[7])
                if code in self.airports_by_codes and code in self.good_codes:
                    self.airports_by_codes[code].set(city, country, x, y)

    def index_cities(self):
        """
        indexes all cities and fills the dictionary of cities and airports
        """
        for code in self.good_codes:
            city = self.airports_by_codes[code].city
            self.airports_by_cities[city] = self.airports_by_cities.get(city, set([]))
            self.airports_by_cities[city].add(code)

    def index_countries(self):
        """
        indexes all countries and fills the dictionary of countries and airports
        """
        for code in self.good_codes:
            c = self.airports_by_codes[code].country
            self.airports_by_countries[c] = self.airports_by_countries.get(c, set([]))
            self.airports_by_countries[c].add(code)

    def suggest_airport(self, string):
        """
        suggest a next airport based on the given string
        :param string: airport code, city name, country, or empty
        :return: airport associated to code, random airport in city or country, otherwise random airport
        """
        # string is an airport code
        if string in self.airports_by_codes:
            return self.airports_by_codes[string]
        # string is a city, then return random airport code for that city
        elif string in self.airports_by_cities:
            airports_in_city = list(self.airports_by_cities[string])
            return self.airports_by_codes[airports_in_city[random.randint(0, len(airports_in_city) - 1)]]
        # string is a country, then return random airport code for that country
        elif string in self.airports_by_countries:
            airports_in_country = list(self.airports_by_countries[string])
            return self.airports_by_codes[airports_in_country[random.randint(0, len(airports_in_country) - 1)]]
        # return random airport code
        else:
            airport_codes = list(self.airports_by_codes.keys())
            return self.airports_by_codes[airport_codes[random.randint(0, len(airport_codes) - 1)]]

Die Überprüfung der Zielbedingung kann bereits vor dem tatsächlichen Enqueueings erfolgen (Zeile 20). Somit wird eine unnötige Speicherung der Knoten vermieden:

In [0]:
def bfs(fr, to):
    """
    implementation of breadth-first search from airport fr to airport to
    :param fr: the starting airport
    :param to: the destination airport
    :return: the found path
    """
    if fr == to:
      raise ValueError("source and destination are equal")
    # FIFO queue of nodes to be processed
    queue = [fr]
    # Avoid loops by keeping track of visited nodes
    visited = {}
    # Store parent of node to get path from fr to to
    parent = {}
    while queue:
        n = queue.pop(0)  # get first node of queue
        visited[n] = True
        for nn in n.connections:  # add successors to queue
            if nn not in visited and nn not in queue:
                parent[nn] = n
                if nn == to:  # no need to append if "to" found -> uses unnecessary memory otherwise
                  print("visited nodes: " + str(len(visited)))
                  return construct_path(parent, fr, to)
                queue.append(nn)  # add nn to the end (important!) of queue (FIFO)

In [0]:
airports = Airports()

print("BFS from Dresden (DRS) to Sydney (SYD)")
start = airports.airports_by_codes["DRS"]
destination = airports.airports_by_codes["SYD"]
print_path(bfs(start, destination))